In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE47727"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE47727"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE47727.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE47727.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE47727.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Global peripheral blood gene expression study [HumanHT-12 V3.0]"
!Series_summary	"Samples were collected from 'control participants' of the Heart and Vascular Health (HVH) study that constitutes a group of population based case control studies of myocardial infarction (MI), stroke, venous thromboembolism (VTE), and atrial fibrillation (AF) conducted among 30-79 year old members of Group Health, a large integrated health care organization in Washington State."
!Series_overall_design	"Total RNA was isolated from peripheral collected using PAXgene tubes and gene expression was profiled using the Illumina platform."
Sample Characteristics Dictionary:
{0: ['age (yrs): 67', 'age (yrs): 54', 'age (yrs): 73', 'age (yrs): 52', 'age (yrs): 75', 'age (yrs): 59', 'age (yrs): 74', 'age (yrs): 76', 'age (yrs): 58', 'age (yrs): 60', 'age (yrs): 66', 'age (yrs): 70', 'age (yrs): 78', 'age (yrs): 77', 'age (yrs): 72', 'age (yrs): 57', 'age (yrs): 63', 'age (yrs): 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see that this dataset contains gene expression data from peripheral blood
# profiled using the Illumina platform. This is not miRNA or methylation data, so gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
# - Age is available at key 0
# - Gender is available at key 1
# - Trait (Arrhythmia/Atrial fibrillation) is not directly available in the sample characteristics,
#   but from the background information, we know these are "control participants" for atrial fibrillation (AF)
#   which means they don't have arrhythmia (value 0)

trait_row = 0  # We'll use the age row as a placeholder for adding our synthetic trait data
age_row = 0    # Age data available at key 0
gender_row = 1 # Gender data available at key 1

# 2.2 Data Type Conversion
# For trait - all subjects are controls (0) for arrhythmia
def convert_trait(input_str):
    return 0  # All subjects are controls (don't have arrhythmia)

# Age conversion function - extract numeric age value
def convert_age(age_str):
    try:
        if age_str and ":" in age_str:
            age_value = age_str.split(":")[1].strip()
            return float(age_value)  # Convert to continuous numeric
        return None
    except:
        return None

# Gender conversion function - convert to binary (0 for female, 1 for male)
def convert_gender(gender_str):
    if gender_str and ":" in gender_str:
        gender_value = gender_str.split(":")[1].strip().lower()
        if 'female' in gender_value:
            return 0
        elif 'male' in gender_value:
            return 1
    return None

# Since we can infer trait data (all subjects are controls), set is_trait_available to True
is_trait_available = True

# 3. Save Metadata
# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since we have inferred trait data, we can perform clinical feature extraction
clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the clinical data
preview_clinical = preview_df(clinical_df)
print("Preview of clinical features:")
print(preview_clinical)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file, index=False)


Preview of clinical features:
{'GSM1298251': [0.0, 67.0, 1.0], 'GSM1298252': [0.0, 54.0, 1.0], 'GSM1298253': [0.0, 73.0, 1.0], 'GSM1298254': [0.0, 52.0, 0.0], 'GSM1298255': [0.0, 75.0, 1.0], 'GSM1298256': [0.0, 59.0, 1.0], 'GSM1298257': [0.0, 74.0, 0.0], 'GSM1298258': [0.0, 75.0, 0.0], 'GSM1298259': [0.0, 74.0, 0.0], 'GSM1298260': [0.0, 76.0, 0.0], 'GSM1298261': [0.0, 73.0, 1.0], 'GSM1298262': [0.0, 67.0, 0.0], 'GSM1298263': [0.0, 58.0, 1.0], 'GSM1298264': [0.0, 60.0, 1.0], 'GSM1298265': [0.0, 66.0, 0.0], 'GSM1298266': [0.0, 70.0, 0.0], 'GSM1298267': [0.0, 75.0, 1.0], 'GSM1298268': [0.0, 70.0, 0.0], 'GSM1298269': [0.0, 78.0, 0.0], 'GSM1298270': [0.0, 77.0, 1.0], 'GSM1298271': [0.0, 72.0, 0.0], 'GSM1298272': [0.0, 78.0, 0.0], 'GSM1298273': [0.0, 57.0, 1.0], 'GSM1298274': [0.0, 77.0, 0.0], 'GSM1298275': [0.0, 63.0, 1.0], 'GSM1298276': [0.0, 62.0, 0.0], 'GSM1298277': [0.0, 52.0, 1.0], 'GSM1298278': [0.0, 74.0, 0.0], 'GSM1298279': [0.0, 59.0, 1.0], 'GSM1298280': [0.0, 64.0, 0.0], 'GSM12982

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE47727/GSE47727_series_matrix.txt.gz


Gene data shape: (48803, 122)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers have the format "ILMN_xxxxx" which are Illumina probe IDs
# These are not human gene symbols but rather probe identifiers specific to Illumina microarray platforms
# These will need to be mapped to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'nuID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILM

Column 'ID' contains values matching gene data ID pattern


Column 'Source' contains values matching gene data ID pattern


Column 'Search_Key' contains values matching gene data ID pattern


Column 'Transcript' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Transcript' may contain gene-related information
Sample values: ['ILMN_44919', 'ILMN_127219', 'ILMN_139282']
Column 'ILMN_Gene' may contain gene-related information
Sample values: ['LOC23117', 'HS.575038', 'FCGR2B']
Column 'Unigene_ID' may contain gene-related information
Sample values: [nan, 'Hs.575038', nan]
Column 'Entrez_Gene_ID' may contain gene-related information
Sample values: [23117.0, nan, 2213.0]
Column 'Symbol' may contain gene-related information
Sample values: ['LOC23117', nan, 'FCGR2B']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns in gene_annotation store probe IDs and gene symbols
probe_id_col = 'ID'  # This contains the ILMN_xxxx identifiers that match gene_data
gene_symbol_col = 'Symbol'  # This contains standard gene symbols

# Verify the column choices by showing a few examples
print(f"Sample probe IDs from gene_annotation['{probe_id_col}']:")
print(gene_annotation[probe_id_col].head(5))
print(f"\nSample gene symbols from gene_annotation['{gene_symbol_col}']:")
print(gene_annotation[gene_symbol_col].head(5))

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"\nGene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(preview_df(mapping_df, n=5))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nGene expression data shape: {gene_data.shape}")
print("First 5 genes:")
print(gene_data.index[:5])

# Write the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Sample probe IDs from gene_annotation['ID']:
0    ILMN_1725881
1    ILMN_1910180
2    ILMN_1804174
3    ILMN_1796063
4    ILMN_1811966
Name: ID, dtype: object

Sample gene symbols from gene_annotation['Symbol']:
0     LOC23117
1          NaN
2       FCGR2B
3       TRIM44
4    LOC653895
Name: Symbol, dtype: object



Gene mapping dataframe shape: (36157, 2)
Gene mapping preview:
{'ID': ['ILMN_1725881', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966', 'ILMN_1668162'], 'Gene': ['LOC23117', 'FCGR2B', 'TRIM44', 'LOC653895', 'DGAT2L3']}



Gene expression data shape: (19120, 122)
First 5 genes:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B'], dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE47727.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (19120, 122)
Gene data shape after normalization: (18326, 122)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE47727.csv


Original clinical data preview:
         !Sample_geo_accession     GSM1298251     GSM1298252     GSM1298253  \
0  !Sample_characteristics_ch1  age (yrs): 67  age (yrs): 54  age (yrs): 73   
1  !Sample_characteristics_ch1   gender: male   gender: male   gender: male   
2  !Sample_characteristics_ch1  tissue: blood  tissue: blood  tissue: blood   

       GSM1298254     GSM1298255     GSM1298256      GSM1298257  \
0   age (yrs): 52  age (yrs): 75  age (yrs): 59   age (yrs): 74   
1  gender: female   gender: male   gender: male  gender: female   
2   tissue: blood  tissue: blood  tissue: blood   tissue: blood   

       GSM1298258      GSM1298259  ...      GSM1298363     GSM1298364  \
0   age (yrs): 75   age (yrs): 74  ...   age (yrs): 71  age (yrs): 73   
1  gender: female  gender: female  ...  gender: female   gender: male   
2   tissue: blood   tissue: blood  ...   tissue: blood  tissue: blood   

      GSM1298365     GSM1298366     GSM1298367     GSM1298368     GSM1298369  \
0  age (y

Data shape after handling missing values: (122, 18329)
Quartiles for 'Arrhythmia':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Arrhythmia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 63.0
  50% (Median): 70.0
  75%: 74.0
Min: 49.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 45 occurrences. This represents 36.89% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (122, 18329)
Dataset is not usable for analysis. No linked data file saved.
